In [10]:
import pandas as pd
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
import numpy as np
import os 
import matplotlib.pyplot as plt 
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler

In [11]:
data_path = '/Users/alessandroquattrociocchi/Documents/Data/pump_station_data/raw_models/'

df = pd.read_parquet(os.path.join(data_path, "pump1_power_siso.par"))
df = df.set_index("time")
df = df.rolling(window=40, center=True).median().dropna()
df = df.resample("1min").mean().dropna()

In [12]:
df['pump1_speed'] = df['pump1_speed'].apply(lambda x: max(0, x))


In [13]:
train_df = df["2024-01-10 06:00:00":"2024-01-10 11:00:00"]
test_df = df["2024-01-11":"2024-02-16 12:00:00"]

In [17]:
from src.sysidentpy import *

In [2]:
!tree

.
├── __init__.py
├── outflow_id_affinity_law.ipynb
├── outflow_id_pwlf.ipynb
├── outflow_id_pwlf2.ipynb
├── outflow_power_id_pwlf.ipynb
├── power1_id_arx copy.ipynb
├── power1_id_arx.ipynb
├── power3_id_arx.ipynb
├── power4_id_arx.ipynb
├── pressure_id_arx.ipynb
└── src
    ├── __pycache__
    │   └── system_identification.cpython-312.pyc
    ├── download_data.py
    ├── preprocessing.py
    ├── static_pumps_models.py
    ├── sysidentpy
    │   ├── __init__.py
    │   ├── __pycache__
    │   │   └── __init__.cpython-312.pyc
    │   ├── basis_function
    │   │   ├── __init__.py
    │   │   ├── _bersntein.py
    │   │   ├── _bilinear.py
    │   │   ├── _fourier.py
    │   │   ├── _hermite.py
    │   │   ├── _hermite_normalized.py
    │   │   ├── _laguerre.py
    │   │   ├── _legendre.py
    │   │   ├── _polynomial.py
    │   │   ├── basis_function_base.py
    │   │   └── tests
    │   │       ├── __init__.py
    │   │       └── test_basis_functions.py
    │   ├── general_estimators
   

In [3]:
!pip install -e .

Obtaining file:///Users/alessandroquattrociocchi/Git/mbs_aggregation_paper/sys_identification
ERROR: file:///Users/alessandroquattrociocchi/Git/mbs_aggregation_paper/sys_identification does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [1]:
from src.system_identification import ARXParametersIdentification

ModuleNotFoundError: No module named 'src.src'

In [2]:
from src.system_identification import ARXParametersIdentification
from src.preprocessing import Preprocessor
import pandas as pd
from src.sysidentpy.basis_function._basis_function import Polynomial

ModuleNotFoundError: No module named 'src.src'

In [ ]:
FEATURES = "pump1_speed"
TARGET = 'pump1_power'

sys_id = ARXParametersIdentification(FEATURES, 
                            TARGET, 
                            dataframe= train_df, 
                            model_type= 'siso')

In [ ]:
parameters = {
    'estimator': 'ridge_regression',
    'ridge_param': 2e6,
    'order_selection': False,
    'n_terms': 3,
    'extended_least_squares': False,
    'ylag': 2,  
    'xlag': 1, 
    'basis_function': Polynomial(degree=1)
}
    
fitted_model, model_parameters = sys_id.identify_arx_parameters(x_train=train_df[FEATURES], 
                                                                y_train= train_df[TARGET],
                                                                **parameters)
    
model_parameters

/home/alqua/Git/PumpStationMPC/sys_identification/src/sysidentpy/narmax_base.py:1034: RuntimeWarning: divide by zero encountered in divide
  x = x[1:] / aux_b
/home/alqua/Git/PumpStationMPC/sys_identification/src/sysidentpy/narmax_base.py:1034: RuntimeWarning: invalid value encountered in divide
  x = x[1:] / aux_b


,Regressors,Parameters,ERR
0,x1(k-1),4.6290E-02,9.7826E-01
1,y(k-1),2.5085E-03,NAN
2,y(k-2),1.6864E-03,NAN


In [ ]:
yhat  = sys_id.predict(x_test= test_df[FEATURES], y_test= test_df[TARGET], n_steps_ahead= 200)

val_plots = sys_id.residuals_analysis(yhat, test_df[TARGET].values)
val_plots
print(model_parameters)

RRSE: 0.1636116971378765
MSE: 17.744120902478016
  Regressors  Parameters         ERR
0    x1(k-1)  4.6290E-02  9.7826E-01
1     y(k-1)  2.5085E-03         NAN
2     y(k-2)  1.6864E-03         NAN


In [ ]:
test_df["yhat"] = yhat

sys_id.plot_features_and_target(test_df, ['yhat', 'pump1_power'])

FigureWidgetResampler({
    'data': [{'name': '<b style="color:sandybrown">[R]</b> yhat <i style="color:#fc9944">~53m</i>',
              'showlegend': True,
              'type': 'scattergl',
              'uid': '58140459-9293-401f-8adf-da1dfee5648c',
              'x': array([datetime.datetime(2024, 1, 11, 0, 0, tzinfo=<UTC>),
                          datetime.datetime(2024, 1, 11, 0, 11, tzinfo=<UTC>),
                          datetime.datetime(2024, 1, 11, 1, 20, tzinfo=<UTC>), ...,
                          datetime.datetime(2024, 2, 16, 9, 43, tzinfo=<UTC>),
                          datetime.datetime(2024, 2, 16, 11, 18, tzinfo=<UTC>),
                          datetime.datetime(2024, 2, 16, 11, 26, tzinfo=<UTC>)], dtype=object),
              'y': array([49.84250018, 47.41934649, 50.19975448, ..., 46.71772651, 48.21408577,
                          47.415006  ])},
             {'name': ('<b style="color:sandybrown">[R' ... 'style="color:#fc9944">~53m</i>'),
              'sh

In [ ]:
yhat  = sys_id.predict(x_test= test_df[FEATURES], y_test= test_df[TARGET], n_steps_ahead= 200)


RRSE: 0.1636116971378765
MSE: 17.744120902478016


In [ ]:
r = pd.read_parquet("/home/alqua/Git/Teaching/Industrial-IoT-For-Digitization-Of-Electronic-Assets-Course/Module 4/code/mpc/eval_result_df.par")

In [ ]:
p1_hat = sys_id.predict(x_test= r["pump1_rpm"], y_test= r[TARGET], n_steps_ahead= 200)


RRSE: nan
MSE: nan


In [ ]:
TARGET

'pump1_power'

In [ ]:
r["pump1_power_hat"] = p1_hat

In [ ]:
r.to_parquet("results_temp_p1.par")